<a href="https://colab.research.google.com/github/elibergman/QM2515-Proyecyo-Final/blob/main/Proyecto_Final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Cargar datos existentes
datos = pd.read_excel('Datos-UV-Visible.xlsx')

# Seleccionar columnas relevantes
X = datos[['Densidad (ppm)', 'Wavelength (nm)']].copy()
y = datos[['Absorbance (AU)', 'Std.Dev.']].copy()

# Preprocesamiento de datos
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.apply(pd.to_numeric, errors='coerce')
X.fillna(X.mean(), inplace=True)

y = y.apply(pd.to_numeric, errors='coerce')
y.fillna(y.mean(), inplace=True)

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Construir un modelo inicial
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_scaled.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')  # Dos nodos de salida lineales
])

# Compilar el modelo
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
modelo.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_absolute_error'])

# Entrenar el modelo con todos los datos existentes
modelo.fit(X_scaled, y, epochs=50, verbose=2)

# Guardar el modelo entrenado
modelo.save('modelo_uv_visible.h5')
print("Modelo guardado")

# Función para cargar el modelo
def cargar_modelo(nombre_archivo):
    return tf.keras.models.load_model(nombre_archivo)

# Cargar el modelo entrenado
modelo_cargado = cargar_modelo('modelo_uv_visible.h5')

# Función para realizar una predicción con el modelo cargado
def predecir_absorbancia_std_cargado(densidad, longitud_onda, modelo):
    entrada = pd.DataFrame({'Densidad (ppm)': [densidad], 'Wavelength (nm)': [longitud_onda]})
    entrada_scaled = scaler.transform(entrada)
    prediccion = modelo.predict(entrada_scaled)
    return prediccion[0]

# Pedir al usuario los valores de densidad y longitud de onda
densidad_usuario = float(input("Ingresa el valor de la densidad (ppm): "))
longitud_onda_usuario = float(input("Ingresa el valor de la longitud de onda (nm): "))

# Realizar la predicción con el modelo cargado
prediccion_cargada = predecir_absorbancia_std_cargado(densidad_usuario, longitud_onda_usuario, modelo_cargado)
print(f"La absorbancia predicha es: {prediccion_cargada[0]}")
print(f"La desviación estándar predicha es: {prediccion_cargada[1]}")

